In [7]:
import subprocess
import time
import threading
import import_ipynb
import ONOS_End_to_End_IBN as onos_ibn
sudo_password = "test@irciss008"

In [8]:
def extract_host_and_ip (flow_data):

    # Extract first flow rule
    flow = flow_data["flows"][0]

    # Extract source and destination IPs
    src_ip = None
    dst_ip = None

    for criterion in flow["selector"]["criteria"]:
        if criterion["type"] == "IPV4_SRC":
            src_ip = criterion["ip"].split("/")[0]  # Remove /32 subnet
        if criterion["type"] == "IPV4_DST":
            dst_ip = criterion["ip"].split("/")[0]

    #print(f"Extracted Source IP: {src_ip}")  # Expected Output: 10.0.0.1 (h1)
    #print(f"Extracted Destination IP: {dst_ip}")  # Expected Output: 10.0.0.4 (h4)

    # Convert IPs to Host Names (Assuming Static Mapping)
    ip_to_host = {
        "10.0.0.1": "h1",
        "10.0.0.2": "h2",
        "10.0.0.3": "h3",
        "10.0.0.4": "h4"
    }

    src_host = ip_to_host.get(src_ip, "Unknown")
    dst_host = ip_to_host.get(dst_ip, "Unknown")

    #print(f"Source Host: {src_host}")  # Expected Output: h1
    #print(f"Destination Host: {dst_host}")  # Expected Output: h4
    return src_host, dst_host, src_ip, dst_ip

In [9]:
def execute_command(command):
    """
    Runs a command with sudo password automation.
    """
    full_command = f"echo {sudo_password} | sudo -S {command}"
    try:
        result = subprocess.run(full_command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            raise Exception(f"Error executing command: {result.stderr.strip()}")
    except Exception as e:
        return str(e)

In [10]:
def execute_command_full(command, timeout):
    """Execute a command and capture full output, waiting for all responses."""
    try:
        process = subprocess.Popen(
            command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )

        output_lines = []
        start_time = time.time()

        while True:
            line = process.stdout.readline()
            if line:
                output_lines.append(line.strip())  # Store the output line
                #print(line.strip())  # Print live output (optional)

            # Check if process has completed
            if process.poll() is not None:
                break

            # Timeout check
            if time.time() - start_time > timeout:
                process.terminate()  # Stop process if timeout occurs
                raise TimeoutError(f"Command timed out after {timeout} seconds")

        # Capture remaining output
        remaining_output, _ = process.communicate()
        if remaining_output:
            output_lines.append(remaining_output.strip())

        return "\n".join(output_lines)

    except Exception as e:
        return str(e)

In [11]:
def onos_assurance (src_host, dst_host, src_ip, dst_ip, ping_count=2):
    
    #host_count_cmd = "ps aux | grep mininet:h | grep root | wc -l"
    #host_count = int(subprocess.check_output(host_count_cmd, shell=True, text=True))-1 #if you want to dynamically find host count
    #print("Total Number of hosts: ",host_count)

    host_pid_cmd = "ps aux | grep mininet:" + src_host + " | awk ' NR==1 { print $2 }'"
    host_pid = int(subprocess.check_output(host_pid_cmd, shell=True, text=True))
    print("\nProcess ID of source host ", src_host,": ", host_pid)

    #dst_host_pid_cmd = "ps aux | grep mininet:" + dst_host  + " | awk ' NR==1 { print $2 }'"
    #dst_host_ip_cmd = "echo " + sudo_password + " | sudo -S mnexec -a $(" + dst_host_pid_cmd + ") ifconfig | grep inet | awk ' NR==1 { print $2 }'"
    #dst_host_ip = execute_command(dst_host_ip_cmd) #If you want to dynamically find a host's IP
    
    dst_host_ip = dst_ip #if you want to use ONOS flow destination IP 
    print("IP of destination host ",dst_host,": ", dst_host_ip)

    timeout=15

    ping_command = f"echo " + sudo_password + f" | sudo -S mnexec -a {host_pid} ping -c {ping_count} {dst_host_ip}"
    #print(execute_command(ping_command))
    output = execute_command_full(ping_command, timeout)
    if "100% packet loss" in output:
        print(f"\nIntent Effective. Traffic from {src_host} to {dst_host} is Blocked.")

    elif "0% packet loss" in output:
        print(f"\nIntent Not Effective. Traffic from {src_host} to {dst_host} is Not BLOCKED.")
    else:
        print("\nIntent Not Effective. Invalid Assurance Report Received. Check command for ping execution.\n")
        print(output)

In [12]:
#onos_assurance('h1', 'h4', '10.0.0.1', '10.0.0.4') #just to test

In [13]:
intent = "Using switch 4, block traffic from 10.0.0.1 to 10.0.0.4."
conflict_status, new_onos_flow_rule = onos_ibn.end_to_end_IBN(intent)
if (conflict_status == 0):
    start_time = time.time()
    src_host, dst_host, src_ip, dst_ip = extract_host_and_ip(new_onos_flow_rule)
    onos_assurance(src_host, dst_host, src_ip, dst_ip)
    elapsed_time = (time.time() - start_time)
    print("Time taken to verify intent: ",  round(elapsed_time,2))


Translated by:  codestral
{"flows": [{"priority": 600, "timeout": 0, "isPermanent": "true", "deviceId": "of:0000000000000004", "selector": {"criteria": [{"type": "ETH_TYPE", "ethType": "0x800"}, {"type": "IPV4_SRC", "ip": "10.0.0.1/32"}, {"type": "IPV4_DST", "ip": "10.0.0.4/32"}]}}]}
Successfully pushed flow rule to ONOS (Device ID: of:0000000000000004, Flow ID: 54043197638039021)
Flow rule exists in ONOS (Flow ID: 54043197638039021)


Successfully translated, conflict checked and installed the rule in ONOS SDN Controller. Time taken:  20.947901725769043

Process ID of source host  h1 :  671393
IP of destination host  h4 :  10.0.0.4

Intent Effective. Traffic from h1 to h4 is Blocked.
Time taken to verify intent:  11.11


In [ ]:
#  timeout=30
#     retries=10
  
#     for attempt in range(retries):
#         result = [None]  # Store result in a mutable object
#         def target():
#             result[0] = execute_command(ping_command)  # Store the function output

#         thread = threading.Thread(target=target)
#         thread.start()
#         thread.join(timeout)  # Wait for the function to complete within the timeout

#         if result[0] is not None:  # If function executed successfully, return result
#             #print(result[0])
#              # Check if traffic is blocked
#             if "0% packet loss" in result[0]:
#                 print(f"\nTraffic from {src_host} to {dst_host} is NOT blocked.")
#                 return
#             elif "100% packet loss" in result[0]:
#                 print(f"\nTraffic from {src_host} to {dst_host} is BLOCKED.")
#                 return
#             else:
#                 #print("\nInvalid Assurance Report Received. Check command for ping execution.")
#                 print(result[0])

#         print(f"Timeout occurred! Retrying... ({attempt+1}/{retries})")
#     raise TimeoutError(f"Function failed to complete after {retries} retries. Check function: execute_command") 